# Reconhecimentos de Padrões

## Alunos: Henrique Ricardo Figueira, Higor Celante
## Dataset: sEMG for Basic Hand movements Data Set


A base de dados utilizada tem as seguintes características:

- Série Temporal
- 2 canais EMG
- 5 arquivos

Datasets:
 - 6s = 2 homens e 2 mulheres na fixa dos 22 anos executaram os 6 movimentos 30 vezes cada, cada movimento sendo coletado por 6s.
 - 5s = 1 homem executou os 6 movimentos 100 vezes cada, por 5s cada movimento.
 
Movimentos:

- Esférico
- Palma aberta
- Lateral
- Cilíndrico
- Gancho
- Pinça


![movimentos](https://i.imgur.com/0faxxKu.png)

### Importações

In [1]:
import scipy.io
import numpy as np
import os
from librosa import stft
from numpy import mean, sqrt, square, arange
from sklearn import svm

#### Carregando o Dataset

In [2]:
caminhos = [os.path.join("src", nome) for nome in os.listdir("src")]
pessoas = []

for essive in caminhos:
    if essive != "/novo.txt":
        mat = scipy.io.loadmat(essive)
        tip = [mat['tip_ch1'], mat['tip_ch2']]
        spher = [mat['spher_ch1'], mat['spher_ch2']]
        palmar = [mat['palm_ch1'], mat['palm_ch2']]
        lateral = [mat['lat_ch1'], mat['lat_ch2']]
        cilindrical = [mat['cyl_ch1'], mat['cyl_ch2']]
        hook = [mat['hook_ch1'], mat['hook_ch2']]
        data = [tip, spher, palmar, lateral, cilindrical, hook]
        data = np.array(data)
        pessoas.append(data)  
        




#print(" 6 movimentos,30 tentativas, 2 canais,  3000 coletas")


### Selecionando 1 dos Datasets

In [3]:
dataone = pessoas[2]
dataone = np.array(dataone)
dataone = np.swapaxes(dataone,1,2)

#### Shape após carregamento

> 6 movimentos, 30 tentativas, 2 canais,  3000 coletas

In [4]:
print(dataone.shape)

(6, 30, 2, 3000)


#### Segmentação

In [5]:
datax = []
segmentosize = (len(dataone[0,0,0,:]))/6
salto = segmentosize * 0.7
antpasso = segmentosize - salto


print("Sobreposicao:        ", antpasso)
print("Salto:               ",salto)
print("Tamanho do segmento: ",segmentosize)


for movimento in dataone:  
    tentativs = []
    for tentativas in movimento:
        canals = []
        for canal in tentativas:
            
            listinha = []
            listinha.append(canal[350:850])
            listinha.append(canal[700:1200])
            listinha.append(canal[1050:1550])
            listinha.append(canal[1400:1900])
            listinha.append(canal[1750:2250])
            listinha.append(canal[2100:2600])
            canals.append(np.split(canal,5) + listinha)
        tentativs.append(canals)
    datax.append( tentativs)
datax = np.array(datax)


Sobreposicao:         150.0
Salto:                350.0
Tamanho do segmento:  500.0


![segmentacao](https://i.imgur.com/rWhhCop.png)


#### Shape após segmentação

##### 6 movimentos, 30 tentativas, 2 canais,  11 segmentos de 500 valores

In [6]:
print(datax.shape)

(6, 30, 2, 11)


## Domínio do tempo

In [75]:
iemg = []
rmslist = []
varlist = []

for movimento in datax:
    listaar = []
    list1 = []
    list2 = []
    for tentativas in movimento:
        canals = []
        for canal in tentativas:
            listseg = 0.0
            for segmento in canal:
                listseg = listseg + (abs(segmento.sum())/ 500 )
            #f = lambda x: np.sum(np.abs(x)) / 500
            rms = sqrt(mean(square(listseg)))
            VAR = np.var(listseg)
            #au = map(f, listseg)
            listaar.append(listseg)
            list1.append(rms)
            list2.append(VAR)
    iemg.append(listaar)
    rmslist.append(listaar)
    varlist.append(list2)

iemg = np.array(iemg)
rmslist = np.array(rmslist)
varlist = np.array(varlist)


### MAV

In [76]:

print(iemg.shape)

(6, 60)


### RMS

In [77]:

print(rmslist.shape)

(6, 60)


### VAR

In [78]:

print(varlist.shape)

(6, 60)


## Domínio da frequência

#### FFT

In [79]:
from scipy.signal import welch

print(datax[:,:,:,0:11].shape)
psdf = []
fmn = []
fmd = []
cont = 0
allft = []
for lvl1 in datax:
    ffttemp = []
    fmd_temp = []
    fmn_temp = []
    som_fmd = 0
    som_fmn = 0
    for lvl2 in lvl1:
        for lvl3 in lvl2:
            #, hop_length=150
            if (cont < 11):
                temp = np.abs(stft(lvl3[cont], n_fft=500,hop_length=350))
                psd = welch(temp)
                psdf.append(psd)
                Fi = (cont * 500) / (2 * len(psd[1]))
                cont+=1
            for lvl3 in psd[1]:
                som_fmd += lvl3.sum()
                som_fmn += lvl3.sum()
                
            fmn_temp.append((Fi * som_fmn) / som_fmn)
            fmd_temp.append(0.5 * som_fmd)
            allfttemp = fmn_temp + fmd_temp
    fmd_temp = np.array(fmd_temp)
    fmd.append(fmd_temp)
    fmn_temp = np.array(fmn_temp)
    fmn.append(fmn_temp)
    allft.append(allfttemp)
    
fmd = np.array(fmd)
fmn = np.array(fmn)
psdf = np.array(psdf)
allft = np.array(allft)
print(psdf.shape)
print(fmd.shape)
print(fmn.shape)
print(allft.shape)

(6, 30, 2, 11)
(11, 2)
(6, 60)
(6, 60)
(6, 120)


#### PSD

In [80]:

print(psdf.shape)

(11, 2)


#### FMD

In [81]:

print(fmd.shape)

(6, 60)


#### FMN

In [82]:

print(fmn.shape)

(6, 60)


#### Junção do FMN e FMD, MAV, RMS, VAR

In [83]:
#allft = allft * VARLIST [:,0:120]

print(allft.shape)

(6, 120)


### Movimento 1

In [84]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OutputCodeClassifier
from sklearn.svm import LinearSVC

In [107]:
teste = np.concatenate((iemg, rmslist, varlist, fmn, fmd), axis =1)
X = teste[0].reshape(-1,1)
X = np.swapaxes(X, 0,1)
y = np.concatenate((teste[1], teste[2], teste[3], teste[4], teste[5]))
y = y[0:300].reshape(-1,1)
y = np.swapaxes(y, 0,1)

In [111]:
#y = []
#for lvl1 in teste:
   #for lvl2 in lvl1:
    #        y.append(lvl2)
#y = np.array(y)
#print(y.shape)

In [114]:
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = OutputCodeClassifier(LinearSVC(random_state=0), code_size=2, random_state=0)

clf.fit(X_train, y_train).predict(X_test) 

(1, 300)
(1, 300)


ValueError: With n_samples=1, test_size=0.33 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [115]:


clf = SVC(gamma='auto')

X = allft
y = ["movimento1", "movimento2", "movimento3", "movimento4", "movimento5", "movimento6"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf.fit(X, y) 


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [117]:
print(clf.predict(X_test))
print(X_test)

['movimento1' 'movimento6']
[[0.00000000e+00 9.96015936e-01 1.99203187e+00 2.98804781e+00
  3.98406375e+00 4.98007968e+00 5.97609562e+00 6.97211155e+00
  7.96812749e+00 8.96414343e+00 9.96015936e+00 9.96015936e+00
  9.96015936e+00 9.96015936e+00 9.96015936e+00 9.96015936e+00
  9.96015936e+00 9.96015936e+00 9.96015936e+00 9.96015936e+00
  9.96015936e+00 9.96015936e+00 9.96015936e+00 9.96015936e+00
  9.96015936e+00 9.96015936e+00 9.96015936e+00 9.96015936e+00
  9.96015936e+00 9.96015936e+00 9.96015936e+00 9.96015936e+00
  9.96015936e+00 9.96015936e+00 9.96015936e+00 9.96015936e+00
  9.96015936e+00 9.96015936e+00 9.96015936e+00 9.96015936e+00
  9.96015936e+00 9.96015936e+00 9.96015936e+00 9.96015936e+00
  9.96015936e+00 9.96015936e+00 9.96015936e+00 9.96015936e+00
  9.96015936e+00 9.96015936e+00 9.96015936e+00 9.96015936e+00
  9.96015936e+00 9.96015936e+00 9.96015936e+00 9.96015936e+00
  9.96015936e+00 9.96015936e+00 9.96015936e+00 9.96015936e+00
  5.97074677e+01 1.00429997e+02 1.79150569